For all orgs that made the cut 1
* we want to enrich them with county statistics
* this requires ein-county mapping
* which requires finding the lat-lon coordinates of an org from address so we can check which county contains the given coordinates using shapefiles (first match is selected)
* which requires using Google Maps GeoCoding API

Geocoding API is expensive so we want to limit the lookups to the orgs that made the first cut and get their unique set of addresses (multiple orgs could be using the same address)





In [ ]:
!pip install googlemaps rtree

In [ ]:
# Native modules
import csv
import time
import zipfile

# External modules
import fiona
import googlemaps
import pandas as pd
import requests
import rtree
import shapely

from google.cloud import bigquery, storage


In [ ]:
API_KEY = "INSERT API KEY"
location = "INSERT LOCATION"
project_id = "INSERT PROJECT ID"
bucket_name = "INSERT BUCKET NAME"
prefix = "irs-form-990"

In [ ]:
gmaps = googlemaps.Client(key=API_KEY)

In [ ]:
client = bigquery.Client(location=location)

In [ ]:
# Identify the list of EINs and their address that require a geocoding lookup
query = """
SELECT ein, CONCAT(TRIM(address), ', ', TRIM(city), ', ', TRIM(state), ', ', zip) AS combined_address
FROM analysis.cut_2_v20240212_filtered
WHERE geo_id IS NULL
AND CAST(ein AS STRING) NOT IN (SELECT DISTINCT ein FROM reference.ein_lat_lon_2022)
"""


In [ ]:
query_job = client.query(query)

In [ ]:
orgs = query_job.to_dataframe()

In [ ]:
orgs.head()

In [ ]:
# Find the geocodes for each organization

with open('orgs_without_coordinates.csv', 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ein', 'latitude', 'longitude'])

    for index, org in enumerate(orgs.itertuples()):
        ein = org.ein
        address = org.combined_address
        geocode = gmaps.geocode(address)
        if len(geocode) == 0:
            lat = None
            lng = None
        else:
            lat = geocode[0]['geometry']['location']['lat']
            lng = geocode[0]['geometry']['location']['lng']

        writer.writerow([ein, lat, lng])

        if index % 10000 == 0:
            print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), index)


In [ ]:
df = pd.read_csv('orgs_without_coordinates.csv')

In [ ]:
df

In [ ]:
# Set EIN to string
df['ein'] = df['ein'].astype(str)

In [ ]:
table_id = "reference.ein_lat_lon_2022"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

job = client.load_table_from_dataframe(df, table_id, job_config=job_config)

job.result()  # Waits for the job to complete

Now we have the coordinates for all EINs, time to find their county using shapefiles

In [ ]:
shape_file_url = "https://www2.census.gov/geo/tiger/TIGER2021/COUNTY/tl_2021_us_county.zip"

In [ ]:
# Download the shape file
r = requests.get(shape_file_url, allow_redirects=True)
open('tl_2021_us_county.zip', 'wb').write(r.content)

In [ ]:
with zipfile.ZipFile('tl_2021_us_county.zip', 'r') as zip_ref:
    zip_ref.extractall('tl_2021_us_county')

In [ ]:
shapefile = fiona.open('tl_2021_us_county/tl_2021_us_county.shp')

In [ ]:
shapefile.schema

In [ ]:
# create a spatial index for the county polygons
index = rtree.index.Index()
for i, county in enumerate(shapefile):
    geometry = shapely.geometry.shape(county['geometry'])
    index.insert(i, geometry.bounds)

In [ ]:
# Get the EINs and their coordinates
ein_lat_lon_query = """
SELECT *
FROM reference.ein_lat_lon_2022
"""

In [ ]:
org_coordinates = client.query(ein_lat_lon_query).to_dataframe()

In [ ]:
org_coordinates.head()

In [ ]:
# find the county that each org is in
ein_county = []
i = 0
for org in org_coordinates.itertuples():
    ein = org.ein
    point = shapely.geometry.Point(org.longitude, org.latitude)
    # if the point is null then skip it
    if point.is_empty:
        continue
    # use the spatial index to find the county that contains the point
    for j in index.intersection(point.bounds):
        county = shapefile[j]
        geometry = shapely.geometry.shape(county['geometry'])
        if geometry.contains(point):
            geo_id = county["properties"]["GEOID"]
            data = {"ein": ein, "geo_id": geo_id}
            ein_county.append(data)
            break
    if i % 10000 == 0:
        print(i, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    i += 1

In [ ]:
ein_county_df = pd.DataFrame(ein_county)

In [ ]:
ein_county_df.head()

In [ ]:
ein_county_df.to_csv("ein_county_2022.csv", index=False)

In [ ]:
storage_client = storage.Client(project=project_id)

In [ ]:
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
blob = bucket.blob(f"{prefix}/reference/ein_county_2022.csv")

In [ ]:
blob.upload_from_filename("ein_county_2022.csv")

In [ ]:
schema = [
    bigquery.SchemaField("ein", "STRING"),
    bigquery.SchemaField("county", "STRING"),
]

In [ ]:
job_config = bigquery.LoadJobConfig(
    schema=schema,
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
)

In [ ]:
table_ref = client.dataset("reference").table("ein_county_2022")

In [ ]:
job = client.load_table_from_uri(
    f"gs://{bucket_name}/{prefix}/reference/ein_county_2022.csv",
    table_ref,
    job_config=job_config,
)



In [ ]:
job.result()